In [1]:
import json
import pygeohash as pgh
import jsonschema
import gzip
import os

# Load the JSON schema from file
with open('C:/Users/chris/DSC650-T301/dsc650/dsc650/assignments/assignment03/schemas/routes-schema.json') as f:
    schema = json.load(f)

# Open the compressed JSONL file
with gzip.open('C:/Users/chris/DSC650-T301/dsc650/data/processed/openflights/routes.jsonl.gz', 'rt') as f:
    # Read each line of the file and parse as JSON
    records = [json.loads(line) for line in f]
records


[{'airline': {'airline_id': 410,
   'name': 'Aerocondor',
   'alias': 'ANA All Nippon Airways',
   'iata': '2B',
   'icao': 'ARD',
   'callsign': 'AEROCONDOR',
   'country': 'Portugal',
   'active': True},
  'src_airport': {'airport_id': 2965,
   'name': 'Sochi International Airport',
   'city': 'Sochi',
   'country': 'Russia',
   'iata': 'AER',
   'icao': 'URSS',
   'latitude': 43.449902,
   'longitude': 39.9566,
   'altitude': 89,
   'timezone': 3.0,
   'dst': 'N',
   'tz_id': 'Europe/Moscow',
   'type': 'airport',
   'source': 'OurAirports'},
  'dst_airport': {'airport_id': 2990,
   'name': 'Kazan International Airport',
   'city': 'Kazan',
   'country': 'Russia',
   'iata': 'KZN',
   'icao': 'UWKD',
   'latitude': 55.606201171875,
   'longitude': 49.278701782227,
   'altitude': 411,
   'timezone': 3.0,
   'dst': 'N',
   'tz_id': 'Europe/Moscow',
   'type': 'airport',
   'source': 'OurAirports'},
  'codeshare': False,
  'equipment': ['CR2']},
 {'airline': {'airline_id': 410,
   'nam

In [2]:
def create_hash_dirs(records):
    # Define the output directory
    geoindex_dir = 'C:/Users/chris/DSC650-T301/dsc650/dsc650/assignments/assignment03/results/geoindex'

    # Create the output directory if it doesn't exist
    if not os.path.exists(geoindex_dir):
        os.makedirs(geoindex_dir)

    # Loop through each record and create a directory based on the geohash
    for record in records:
        src_airport = record.get('src_airport')
        if src_airport:
            latitude = src_airport.get('latitude')
            longitude = src_airport.get('longitude')
            if latitude is not None and longitude is not None:
                # Generate the geohash from the latitude and longitude
                geohash = pgh.encode(latitude, longitude)
                # Create the directory structure based on the first two characters of the geohash
                dir1 = os.path.join(geoindex_dir, geohash[0])
                dir2 = os.path.join(dir1, geohash[0:2])

                if not os.path.exists(dir2):
                    os.makedirs(dir2)

                # Write the record to a JSON file named after the geohash
                file_path = os.path.join(dir2, geohash[0:2] + '.jsonl.gz')
                with open(file_path, 'a') as f:
                    f.write(json.dumps(record) + '\n')

In [3]:
create_hash_dirs(records)

In [12]:
import gzip
import json
import pygeohash as pgh

def geospatial_search(latitude, longitude, max_distance_km):
    # Define an empty list to store matching airports
    matching_airports = []

    # Open the routes JSONL file using gzip
    with gzip.open('C:/Users/chris/DSC650-T301/dsc650/data/processed/openflights/routes.jsonl.gz', 'rt') as f:
        # Read each line of the file and parse as JSON
        records = [json.loads(line) for line in f]

        # Loop over each record
        for record in records:
            src_airport = record.get('src_airport')
            if src_airport:
                # Get the latitude and longitude of the source airport
                airport_latitude = src_airport.get('latitude')
                airport_longitude = src_airport.get('longitude')
                if airport_latitude is not None and airport_longitude is not None:
                    # Calculate the geohash distance between the input coordinates and the airport coordinates
                    input_geohash = pgh.encode(latitude, longitude)
                    airport_geohash = pgh.encode(airport_latitude, airport_longitude)
                    distance_km = pgh.geohash_approximate_distance(input_geohash, airport_geohash) / 1000
                    #print(f"Distance between {input_geohash} and {airport_geohash}: {distance_km} km")
                    if distance_km <= max_distance_km:
                        # If the distance is within the maximum specified distance, add the airport to the list of matching airports
                        matching_airports.append(src_airport)
    #print(len(matching_airports))
    #print(matching_airports)
    return matching_airports



In [14]:
# Find airports within 20 kilometers of latitude 37.7749, longitude -122.4194
nearby_airports = geospatial_search(41.1499988, -95.91779, 20)

# Print the name of each nearby airport without duplicates
printed_airport_names = set()
for airport in nearby_airports:
    name = airport['name']
    if name not in printed_airport_names:
        print(name)
        printed_airport_names.add(name)

Eppley Airfield
